#  基于MindSpore构建Max Pooling层
本实验主要专注于Max Pooling层的构建原理，利用MindSpore中的nn模块构建Max Pooling层。
## 1 实验目的
1.通过实验了解Max Pooling层的计算原理

2.基于MindSpore中的nn模块实现Max Pooling层的构建
## 2 Max Pooling原理介绍
进行卷积操作后，需要Max Pooling操作进行最大池化，我们用下图进行演示:将整个图片不重叠的分割成多个大小一致的小块（pooling size）,每个小块内仅取最大数字，舍弃其他区节点，保持原有的平面结构，输出最后结果。
![jupyter](./Figures/fig001.png)
Max Pooling在不同的depth上是分开执行的，且不需要参数控制。Max pooling的主要功能是缩减像素采样，但是Max pooling对识别结果有所损坏， 即卷积后的特征图像中会有冗余信息。我们需要反向思考，这些冗余信息从何而来。
为探测到某个特定形状的存在，用一个filte对整个图片逐步扫描。然而只得到了该特定形状的区域卷积的输出，这个输出结果对我们有用，进而用该fiter卷积其他区域，此时，得出的数值可以判定该形状是否存在，并且准确率很高。比如下图中：
![jupyter](./Figures/fig002.png)
考虑探测“横折”这个形状：进行卷积操作后得到3x3的特征图像，这些图像中真正有用的是数字为“3”的节点，其余数值对于此任务结果没有影响。所以用3x3的Max Pooling后，并没有对“横折”的探测产生影响。如果不使用Max Pooling，而让网络自己去学习，网络也会去学习与Max Pooling近似效果的权重。因为是效果近似，增加了更多的参数，其效果反而可能比Max Pooling差。
## 3 实验环境
### 实验环境要求
在动手进行实践之前，需要注意以下几点：
* 确保实验环境正确安装，包括安装MindSpore。安装过程：首先登录[MindSpore官网安装页面](https://www.mindspore.cn/install)，根据安装指南下载安装包及查询相关文档。同时，官网环境安装也可以按下表说明找到对应环境搭建文档链接，根据环境搭建手册配置对应的实验环境。
* 推荐使用交互式的计算环境Jupyter Notebook，其交互性强，易于可视化，适合频繁修改的数据分析实验环境。
* 实验也可以在华为云一站式的AI开发平台ModelArts上完成。
* 推荐实验环境：MindSpore版本=MindSpore 2.0；Python环境=3.7


|  硬件平台 |  操作系统  | 软件环境 | 开发环境 | 环境搭建链接 |
| :-----:| :----: | :----: |:----:   |:----:   |
| CPU | Windows-x64 | MindSpore2.0 Python3.7.5 | JupyterNotebook |[MindSpore环境搭建实验手册第二章2.1节和第三章3.1节](./MindSpore环境搭建实验手册.docx)|
| GPU CUDA 10.1|Linux-x86_64| MindSpore2.0 Python3.7.5 | JupyterNotebook |[MindSpore环境搭建实验手册第二章2.2节和第三章3.1节](./MindSpore环境搭建实验手册.docx)|
| Ascend 910  | Linux-x86_64| MindSpore2.0 Python3.7.5 | JupyterNotebook |[MindSpore环境搭建实验手册第四章](./MindSpore环境搭建实验手册.docx)|

## 4 数据处理
### 4.1数据准备
数据集的获取：

这里使用随机给出的测试数据：

（1）数据维度x=[1,2,4]，并设置kernel_size=3, 步长stride=1，其中数值范围在0到10之间，在后续需要使用MindSpore的tensor进行转换，用于构建MaxPool1d层。

（2）数据维度y=[1, 2, 4, 4]，并设置池化层kernel_size=3，步长stride=1，其中数值范围在0到10之间，在后续需要使用MindSpore的tensor进行转换，用于构建MaxPool2d层。
### 4.2数据加载
模型构建部分加载数据与模型测试部分加载数据

In [2]:
#导入mindspore
import mindspore
#导入numpy库
import numpy as np
#导入mindspore中的nn模块
import mindspore.nn as nn
#从MindSpore中导入Tensor库
from mindspore import Tensor
#模型构建部分加载数据
x= Tensor(np.random.randint(0, 10, [1, 2, 4]), mindspore.float32)
#模型测试部分加载数据
y= Tensor(np.random.randint(0, 10, [1, 2, 4, 4]), mindspore.float32)
print("x:")
print(x)
print("y:")
print(y)

x:
[[[8. 5. 3. 8.]
  [2. 9. 0. 8.]]]
y:
[[[[6. 8. 0. 7.]
   [1. 7. 6. 9.]
   [1. 5. 6. 9.]
   [3. 2. 5. 3.]]

  [[2. 1. 0. 4.]
   [3. 8. 1. 8.]
   [8. 1. 5. 0.]
   [1. 7. 1. 8.]]]]


## 5 模型构建
需要使用numpy库以及MindSpore中的nn模块进行最大池化层的构建，并利用MindSpore中的Tensor进行数据转换。使用nn模块构建MaxPool1d池化层，设置池化层kernel_size=3, 步长stride=1，根据给定的参数建立MaxPool1d池化层。输入数据维度x=[1, 2, 4]，利用Tensor进行数据转换，其中数值范围在0到10之间，并给出池化后的输出结果。

代码如下：

In [3]:
#定义池化层大小与步长
max_pool = nn.MaxPool1d(kernel_size=3, stride=1)
#输入数据维度，并用Tensor将其转化为32浮点数
x= Tensor(np.random.randint(0, 10, [1, 2, 4]), mindspore.float32)
#经过池化后，输出数据维度
output = max_pool(x)
result = output.shape
print(result)

(1, 2, 2)


## 6 模型测试
根据步骤1到步骤3，使用nn模块构建MaxPool2d池化层，输入数据维度y=[1, 2, 4, 4]，数据需要利用Tensor进行数据转换，并设置池化层kernel_size=3, 步长stride=1，其中数值范围在0到10之间,并给出池化后的输出结果。

提示：

步骤一：设置池化层kernel_size=3, 步长stride=1；

步骤二：输入数据维度，并用Tensor将其转化为32浮点数；

步骤三：经过池化后，输出数据维度。

请根据提示，补充完整代码：

In [4]:
#参考答案
#设置池化层kernel_size=3, 步长stride=1
pool = nn.MaxPool2d(kernel_size=3, stride=1)
#输入数据维度，并用Tensor将其转化为32浮点数
y= Tensor(np.random.randint(0, 10, [1, 2, 4, 4]), mindspore.float32)
#经过池化后，输出数据维度
output = pool(y)
print(output.shape)

(1, 2, 2, 2)


## 7 实验总结
本实验介绍了如何基于MindSpore构建Max Pooling层。实验在两个不同的数据维度与参数的基础上，利用MindSpore建立了两个不同的池化层，分别是MaxPool1d层、MaxPool2d层，并对池化后的结果进行输出。